In [ ]:
# Decorations
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import gmplot
from config import api_key
from pprint import pprint

In [ ]:
# Import database
df1 = pd.read_csv('Data/housing.csv')
df2 = pd.read_csv('Data/Total.csv')
df1 = df1.dropna()

### df1(1990) Gmap Plotting

Sort and slice data to get top and bottom 5%

In [ ]:
# SSort data by house value
df1 = df1.sort_values('median_house_value', ascending=False).reset_index(drop=True)
df1.head()

In [ ]:
# Slice the top 5%
top5 = df1.iloc[:round(len(df1['median_house_value'])*0.05), :].sort_values('median_house_value', ascending=True)

In [ ]:
# slice the bottom 5%
bottom5 = df1.iloc[round(len(df1['median_house_value'])*0.95): len(df1['median_house_value']),:].sort_values('median_house_value', ascending=True)

Plotting Top 5% and Bottom 5% on gmap (Use gmplot package)

In [ ]:
# Set the center of the map
gmap_df1 = gmplot.GoogleMapPlotter(df1['latitude'].median(),
                                   df1['longitude'].median(), 100000)
# Plot scatter points based on LatLng
gmap_df1.scatter(df1['latitude'], df1['longitude'], '#FF0000', 
                              size = 2000, marker = False ) 
# Draw out to 'Plot' folder in html format
gmap_df1.draw("Plot/gmap_df1.html")

In [ ]:
# Set the center of the map
gmap = gmplot.GoogleMapPlotter(top5['latitude'].median(),
                                   top5['longitude'].median(), 100000)
# Plot scatter points based on LatLng
gmap.scatter(top5['latitude'], top5['longitude'], '#FF0000', 
                              size = 2000, marker = False ) 
gmap.scatter(bottom5['latitude'], bottom5['longitude'], '#110870', 
                              size = 2000, marker = False ) 
# Draw out to 'Plot' folder in html format
gmap.draw("Plot/gmap.html")

### Getting LatLng using geocoder API for df2

In [ ]:
# Keep only the columns that are useful 
df2 = df2[['Sub Type', 'St#', 'St Name', 'City', 'L/C Price', 'Br/Ba', 'YrBuilt']]
df2.head()

In [ ]:
yr_blt = df2.loc[:, 'YrBuilt'].str.split('/', expand=True)[0]
yr_blt = pd.DataFrame(yr_blt)
yr_blt = yr_blt.fillna(0)

In [ ]:
df2['Age'] = ''
count = 0
for i in yr_blt[0]:
        i = 2018 - int(i)
        df2['Age'][count] = i
        count += 1
df2.head()

In [ ]:
df2['house_price'] = df2.loc[:, 'L/C Price'].str.split('$', expand=True)[1] 

In [ ]:
df2.head()

In [ ]:
df2_by_value = df2.set_index("L/C Price")
df2_by_value = df2_by_value.reset_index()
df2_by_value.head()

In [ ]:
# import CityCode
citycode_df = pd.read_csv('Data/CleanCityCode_nospace.csv')
citycode_df = citycode_df[['City', 'Code']]
citycode_df['Code'] = citycode_df['Code'].str.strip()

In [ ]:
df2_by_value['City Name'] = ""
for index, row in df2_by_value.iterrows():
    for i, r in citycode_df.iterrows():
        if r['Code'] == row['City']:
            row['City Name'] = r['City']
            if index%200 == 0:
                print(index)

In [ ]:
df2_by_value["Address"] = df2_by_value["St#"].astype(str) + " " + df2_by_value["St Name"].astype(str) + " " + df2_by_value['City Name']
df2_by_value['Lat'] = ""
df2_by_value['Lng'] = ""

In [ ]:
params = {"key": api_key}
for index, row in df2_by_value.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params['address'] = row['Address']
    geo_data = requests.get(base_url, params).json()
    try:
        df2_by_value["Lat"][index] = geo_data["results"][0]["geometry"]["location"]["lat"]
        df2_by_value["Lng"][index] = geo_data["results"][0]["geometry"]["location"]["lng"]
        if index%100 == 0:
            print(index)
        if index%500 == 0:
            print('Whats up bro, waiting too long? Call me at (949)111-1111')
    except IndexError:
        print(f"Row {index} cannot be found on gmap.")
        continue

In [ ]:
# Output to CSV
df2_by_value.to_csv("Data/df2_latlng.csv")

### df2(2017) Gmap Plotting

In [ ]:
df2_latlng = pd.read_csv("Data/df2_latlng.csv")

In [ ]:
df2_latlng = df2_latlng[['L/C Price','Lat', 'Lng']].dropna()

In [ ]:
top5_df2 = df2_latlng.iloc[:round(len(df2_latlng['L/C Price'])*0.05), :].sort_values('L/C Price', ascending=True)
bottom5_df2 = df2_latlng.iloc[round(len(df2_latlng['L/C Price'])*0.95): len(df2_latlng['L/C Price']),:].sort_values('L/C Price', ascending=True)

In [ ]:
# Set the center of the map
gmap_df2_tab = gmplot.GoogleMapPlotter(top5_df2['Lat'].median(),
                                   top5_df2['Lng'].median(), 100000)
# Plot scatter points based on LatLng
gmap_df2_tab.scatter(top5_df2['Lat'], top5_df2['Lng'], '#FF0000', 
                              size = 2000, marker = False ) 
gmap_df2_tab.scatter(bottom5_df2['Lat'], bottom5_df2['Lng'], '#110870', 
                              size = 2000, marker = False ) 
# Draw out to 'Plot' folder in html format
gmap_df2_tab.draw("Plot/gmap_df2_tab.html")

In [ ]:
# Set the center of the map
gmap_df2 = gmplot.GoogleMapPlotter(df2_latlng['Lat'].median(),
                                df2_latlng['Lng'].median(), 100)
# Plot scatter points based on LatLng
gmap_df2.scatter(df2_latlng['Lat'], df2_latlng['Lng'], '#FF0000', 
                              size = 1000, marker = False) 
# Draw out to 'Plot' folder in html format
gmap_df2.draw("Plot/gmap_df2.html")